
- 예약 시간
 - 10분간격, 30분 간격, 1시간 간격으로 변환하여 예상해보기

- 주변 교통량(나중에)
 - 넣고 안넣고 비교해보기(지표를 좀 더 고려해보기)
 - (서울시 교통량 이력 정보 : 교통량변수와 차로번호, 그 외 참고자료도 찾아보기)
- 요일과 주말 
 - 참고 PPT에서는 비슷하다고 했지만 양상이 너무 다름
-출발지, 목적지의 언급 숫자(나중)
-날씨(추후 추가)
-시간대별 대기시간(target)

### 예약 시간, 요일(주말), 출발지_목적지 언급, 시간대별 대기시간


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 기본 데이터 가져오기
%cd /content/drive/MyDrive/'멀티캠퍼스 5조'/2.데이터/일일이용데이터

df = pd.read_csv('2019-2022년_서울시.csv', parse_dates=['예정일시', '배차일시', '승차일시'], infer_datetime_format=True)

/content/drive/.shortcut-targets-by-id/1Mz49-WiynxJxHmmXYuWSqduV3-jd-BC7/멀티캠퍼스 5조/2.데이터/일일이용데이터


In [ ]:
# 필요없는 열 제거하기
df = df[['출발지구군', '출발지상세', '예정일시', '승차일시']]

In [ ]:
# 인접한 동 리스트 반환
%cd /content/drive/MyDrive/멀티캠퍼스 5조/2.데이터
json = json.load(open('./json/서울행정동_수정.json','r', encoding = 'utf-8'))
features = pd.DataFrame(json['features'])
features = features[['properties', 'geometry']]

# 같은 점을 공유하는 행정동의 이름을 dict에 넣기.
features_dict = {}
for num in tqdm(features.index):
    adm_nm = []                                                             # 겹치는 동을 넣어줄 빈 리스트
    key = features.loc[num,'properties']['동']                              # 기준이 되는 동
    lst1 = features.loc[num, 'geometry']['coordinates'][0][0]               # 기준이 되는 동이 사용하는 점들의 list
    features2 = features[features['properties'] !=features.iloc[num,0]]     # 기준 동을 제외한 비교용 데이터프레임
    for i in features2.index:
        lst2 = features2.loc[i, 'geometry']['coordinates'][0][0]
        if len(np.intersect1d(np.array(lst1), np.array(lst2))) > 0:
            adm_nm.append(features2.loc[i,'properties']['동'])
    features_dict[key] = adm_nm

# print(features_dict)

/content/drive/.shortcut-targets-by-id/1Mz49-WiynxJxHmmXYuWSqduV3-jd-BC7/멀티캠퍼스 5조/2.데이터


100%|██████████| 423/423 [00:35<00:00, 12.02it/s]


#### 데이터 통합에 사용되는 보조 함수, 코드

In [ ]:
# 시간대 나누기
def time_seperate(x):
  # std는 기준, 분단위
  return_value = x.hour * 2
  if x.minute < 30:
    return return_value
  else:
    return return_value + 1

In [ ]:
def adjacent_area(x, feature_dict):
  # x와 인접한 지역을 찾아 인접지역+x리스트 반환하는 함수
  area_list = [x]
  area_list.extend(feature_dict[x])
  return area_list

In [ ]:
# 지역, 시간 별 언급 횟수 파일 만들고 불러오기

# my_precious = pd.DataFrame()
# area = []
# clock = []
# count = []
# for key in tqdm(features_dict):
#   for time in range(48):
#     area.append(key)
#     clock.append(time)
#     count.append(len(df[(df['출발지상세'].isin(adjacent_area(key, features_dict))) & (df['시간대(30분)'] == time)]))

# my_precious['지역'] = area
# my_precious['시간'] = clock
# my_precious['횟수'] = count
%cd /content/drive/MyDrive/멀티캠퍼스 5조/2.데이터
# my_precious.to_csv('지역_시간별 언급횟수.csv', index=False)
mentions = pd.read_csv('지역_시간별 언급횟수.csv')
mention = mentions.groupby(['지역', '시간']).sum()

/content/drive/.shortcut-targets-by-id/1Mz49-WiynxJxHmmXYuWSqduV3-jd-BC7/멀티캠퍼스 5조/2.데이터


In [7]:
# 교통량 데이터 가져오기
%cd /content/drive/MyDrive/'멀티캠퍼스 5조'/2.데이터/'교통량 데이터'
road = pd.read_csv('2019-2022도로속도 결측값 처리-0512 진짜.csv')

road['일자'] = road['일자'].str.replace('.', '-')
road['index'] = road.apply(lambda x: x['일자'] + ' ' + x['자치구명'], axis=1)
road.set_index('index', inplace=True)
del road['Unnamed: 0']
del road['일자']
del road['자치구명']

columns = [str(i) for i in range(48)]
road.columns = columns
road.head(3)

/content/drive/.shortcut-targets-by-id/1Mz49-WiynxJxHmmXYuWSqduV3-jd-BC7/멀티캠퍼스 5조/2.데이터/교통량 데이터


,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
index,,,,,,,,,,,,,,,,,,,,,
2019-01-01 종로구,25.25,25.25,25.90,25.90,29.18,29.18,30.90,30.90,31.69,31.69,...,25.9,25.9,27.01,27.01,26.90,26.90,28.33,28.33,29.06,29.06
2019-01-01 중구,23.20,23.20,24.08,24.08,27.31,27.31,28.35,28.35,28.63,28.63,...,23.1,23.1,24.27,24.27,24.90,24.90,26.15,26.15,26.79,26.79
2019-01-01 용산구,28.96,28.96,28.39,28.39,29.97,29.97,32.54,32.54,33.83,33.83,...,30.6,30.6,31.23,31.23,31.96,31.96,32.95,32.95,34.52,34.52


In [ ]:
# 날씨 데이터 가져오기
%cd /content/drive/MyDrive/'멀티캠퍼스 5조'/2.데이터/'날씨 데이터'
weather = pd.read_csv('2019-2022년_날씨.csv',parse_dates=['시간'], infer_datetime_format=True)
del weather['Unnamed: 0']
weather.dropna(inplace=True)
weather.rename(columns={"시간":"승차일시"}, inplace=True)

/content/drive/.shortcut-targets-by-id/1Mz49-WiynxJxHmmXYuWSqduV3-jd-BC7/멀티캠퍼스 5조/2.데이터/날씨 데이터


#### 데이터 통합

In [34]:
# %cd /content/drive/MyDrive/멀티캠퍼스 5조/2.데이터
# df = pd.read_csv('학습데이터.csv', parse_dates=['예정일시', '승차일시'], infer_datetime_format=True)

# 예약 시간
# df['시간대(30분)'] = df['예정일시'].apply(lambda x: time_seperate(x))

# 주변 교통량
# df['평균속도'] = df.apply(lambda x: road.loc[str(x['예정일시'].date()) + ' ' + x['출발지구군'], str(x['시간대(30분)'])], axis=1)

# 요일과 주말
# df['요일'] = df['승차일시'].apply(lambda x: x.weekday())

# 월화수목금토일
# for index, weekday in enumerate('월 화 수 목 금 토 일'.split()):
#   df[weekday] = df['요일'].apply(lambda x: 1 if x == index else 0)

# 출발지, 목적지의 언급량
# df['언급량'] = df.apply(lambda x: mention.loc[(x['출발지상세'], x['시간대(30분)'])].iloc[0], axis=1)

# 날씨
# df = pd.merge_asof(
#     left = df.sort_values(by='승차일시'),
#     right = weather.sort_values(by='승차일시'),
#     on = '승차일시',
#     direction = 'nearest',
# )

# 대기시간(target)
# df['대기시간(초)'] =  (df['승차일시'] - df['예정일시']).apply(lambda x: x.total_seconds())

df.to_csv('학습데이터.csv', index=False)

In [33]:
df.head(3)

,출발지구군,출발지상세,예정일시,승차일시,시간대(30분),요일,월,화,수,목,금,토,일,언급량,대기시간,대기시간(초),강수량,적설량,평균속도
0,은평구,구산동,2019-01-01 00:21:00,2019-01-01 02:13:11,0,1,0,1,0,0,0,0,0,48,112.0,6731.0,0.0,0.0,25.59
1,강북구,인수동,2019-01-01 00:15:00,2019-01-01 03:23:48,0,1,0,1,0,0,0,0,0,55,188.0,11328.0,0.0,0.0,24.24
2,강남구,일원1동,2019-01-01 00:49:00,2019-01-01 03:34:24,1,1,0,1,0,0,0,0,0,33,165.0,9924.0,0.0,0.0,30.75
